<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto tem o intuito de estudar Técnicas de Modelagem de Risco de Crédito. </i>       
</p>  

# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [1]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.tsa import stattools
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import bernoulli, binom, poisson, geom, norm, chi2, f, normaltest, ttest_ind, ttest_rel, wilcoxon, mannwhitneyu, kruskal
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px 
import plotly.graph_objects as go
import importlib
import transition_matrix_estimator
from transition_matrix_estimator import TransitionMatrixLearner
import time
import datetime
import warnings
import category_encoders as ce 
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix

%matplotlib inline
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='orange' style='font-size: 40px;'> Exemplo Montagem de Target </font>
<hr style='border: 2px solid orange;'>

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

> 1. Testes de Continuidade: Verificação da consistência temporal dos dados por contrato (sem quebras ou lacunas indevidas).

> 2. Criação da Coluna flag_acordo: Identificação de registros com acordos ou renegociações.

> 3. Criação da Coluna over90: Flag mensal de inadimplência grave (atraso superior a 90 dias).

> 4. Criação da Coluna mau: Identificação de contratos irrecuperáveis com base em regras de negócio.

> 5. Construção do Grupo Performing: Definição da amostra elegível para modelagem de PD, com base no status performing.

> 6. Criação da Coluna ever90m12: Indica se o contrato atingiu atraso >90 dias em qualquer mês dos próximos 12 meses (sem considerar acordos).

> 7. Criação da Coluna over90m12: Indica se o contrato está em atraso >90 dias ao final da janela futura de 12 meses (sem considerar acordos).

> 8. Criação da Coluna target: Flag final de inadimplência futura, considerando tanto atraso quanto acordos.

> 9. Congelamento do Snapshot de Contratos: Seleção de uma linha por contrato para representar o momento de avaliação (janela de decisão).

> 10. Curva de Inadimplência (over90) por Mês: Análise temporal do volume de contratos inadimplentes mês a mês (painel e snapshot).

> 11. Curvas de ever90m12 e over90m12 por Mês: Evolução mensal dos indicadores prospectivos de risco (apenas painel).

> 12. Construção da Matriz de Transição: Avaliação da migração de status de crédito entre períodos (ex: performing → inadimplente).

> 13. Construção da Curva de Cura: Medição da recuperação de contratos inadimplentes ao longo do tempo.

### 1.1.1) Entendimento de Conceitos

> 1. data_inicio_contrato

- É a data que o cliente e o banco firmaram acordo, como por exemplo 2014-09-30
- Em muitos casos, existe "carência" até o 1 vencimento, portanto, o contrato existe juridicamente mas não dá para medir performance ainda

> 2. safra:

- Mês de nascimento da performance (M0)
- Se a primeira parcela vence em jan/2025, o contrato é marcado como safra de 201501

> 3. dat_ref:

- A cada dat_ref (foto mensal), o contrato é monitorado
- A safra nunca muda, porque é o "carimbo" da geração

In [57]:
historico  = pd.read_parquet('./data/full_history.parquet')
historico['safra'] = historico.groupby('id_contrato')['data_ref'].transform('min')
historico = historico[['id_contrato','data_inicio_contrato', 'safra', 'data_ref','dias_atraso']]
historico.sort_values(by=['id_contrato','data_ref'], inplace=True)
historico['data_ref'] = pd.to_datetime(historico['data_ref'])


rastro_contratos = pd.read_parquet('./data/rastro_contratos.parquet')
rastro_contratos.sort_values(by=['id_antigo'], inplace=True)
rastro_contratos['data_evento'] = pd.to_datetime(rastro_contratos['data_evento'])
rastro_contratos.rename(columns={'id_antigo': 'id_contrato', 'data_evento': 'data_inicio_contrato'}, inplace=True)

display(historico.head(15))
display(rastro_contratos.head(15))



,id_contrato,data_inicio_contrato,safra,data_ref,dias_atraso
0,10000000,2014-09-30,2015-01-01,2015-01-01,30
1,10000000,2014-09-30,2015-01-01,2015-02-01,15
2,10000000,2014-09-30,2015-01-01,2015-03-01,15
3,10000000,2014-09-30,2015-01-01,2015-04-01,30
4,10000000,2014-09-30,2015-01-01,2015-05-01,60
5,10000000,2014-09-30,2015-01-01,2015-06-01,90
6,10000000,2014-09-30,2015-01-01,2015-07-01,60
7,10000000,2014-09-30,2015-01-01,2015-08-01,90
8,10000000,2014-09-30,2015-01-01,2015-09-01,90
9,10000000,2014-09-30,2015-01-01,2015-10-01,60


,id_contrato,id_novo,data_inicio_contrato
1023,10000000,10004583,2015-12-01
368,10000001,10003338,2015-06-01
706,10000002,10003996,2015-09-01
0,10000003,10002500,2015-02-01
255,10000007,10003118,2015-05-01
1366,10000009,10005220,2016-03-01
815,10000010,10004198,2015-10-01
55,10000011,10002680,2015-03-01
816,10000013,10004199,2015-10-01
256,10000015,10003119,2015-05-01


### 1.1.2) Flag de Renegociação   

> 1. Renegociação é quando o cliente não consegue mais horar o contrato original e o banco faz um novo acordo para tentar recuperar o crédito. Exemplo:

- O cliente estava atrasado várias parcelas
- Ao invés do banco contabilizar uma perda efetivamente, o banco oferece uma renegociação:
    - Pode reparcelar o saldo devedor em novas condições de prazos
    - Pode descontar parte da dívida
    - Pode dar carência antes de retomar os pagamentos
- Nesses casos, o ID do contrato geralmente muda e o novo ID carrega a dívida renegociada

In [58]:
historico['flag_acordo'] = 0

rastro_contratos["safra"] = np.nan
rastro_contratos["data_ref"] = np.nan
rastro_contratos["dias_atraso"] = np.nan
rastro_contratos['flag_acordo'] = 1
rastro_contratos = rastro_contratos[['id_contrato', 'data_inicio_contrato', 'safra', 'data_ref', 'dias_atraso', 'flag_acordo']]

df = pd.concat([historico, rastro_contratos])
df.sort_values(by=['id_contrato','data_ref'], inplace=True)
df = df.reset_index(drop=True)
df.head(15)

,id_contrato,data_inicio_contrato,safra,data_ref,dias_atraso,flag_acordo
0,10000000,2014-09-30,2015-01-01,2015-01-01,30.0000,0
1,10000000,2014-09-30,2015-01-01,2015-02-01,15.0000,0
2,10000000,2014-09-30,2015-01-01,2015-03-01,15.0000,0
3,10000000,2014-09-30,2015-01-01,2015-04-01,30.0000,0
4,10000000,2014-09-30,2015-01-01,2015-05-01,60.0000,0
5,10000000,2014-09-30,2015-01-01,2015-06-01,90.0000,0
6,10000000,2014-09-30,2015-01-01,2015-07-01,60.0000,0
7,10000000,2014-09-30,2015-01-01,2015-08-01,90.0000,0
8,10000000,2014-09-30,2015-01-01,2015-09-01,90.0000,0
9,10000000,2014-09-30,2015-01-01,2015-10-01,60.0000,0


### 1.1.3) Criação de Mau Origem, Over e Ever

> 1. Desconsideraremos a Flag de Acordo pois eu só conseguiria marcar Mau Origem, Over e Ever mediante a saber quantos dias de atraso o novo contrato possui

> 2. Em termos práticos:

| Tipo     | Uso         | Exemplo                                                         |
| -------- | ----------- | --------------------------------------------------------------- |
| Snapshot | Application | “Este cliente, até o mês corrente, já atrasou >30 dias?”        |
| Painel   | Behaviour   | “Este cliente, em algum momento do contrato, atrasou >30 dias?” |


In [59]:
df = pd.concat([historico, rastro_contratos])
df = df.loc[df['flag_acordo'] == 0]
df.sort_values(by=['id_contrato','data_ref'], inplace=True)
df.head(15)

# 1) Definir safra e idade em meses
df['idade_meses_contrato'] =df['data_ref'].dt.to_period('M').astype(int) - df['safra'].dt.to_period('M').astype(int)

# ==========================
# MAU_ORIGEM (exemplo: >30 dias em M0)
# ==========================

df['flag_mau_origem'] = np.where((df['dias_atraso'] >= 30) & (df['idade_meses_contrato'] == 0), 1, 0)
df['mau_origem'] = df.groupby('id_contrato')['flag_mau_origem'].transform('max')
df.drop(columns=['flag_mau_origem'], inplace=True)


# --------------------------
# EVER30M6
# --------------------------

# 1) Linha a linha → 1 apenas nas referências que bateram o limiar
df['ever30m6_snapshot'] = np.where((df['dias_atraso'] >= 30) & (df['idade_meses_contrato'] <= 6),1, 0)

# 2) Contrato inteiro → 1 se em algum mês bateu o limiar
df['ever30m6_painel'] = df.groupby('id_contrato')['ever30m6_snapshot'].transform('max')

# --------------------------
# EVER30M12
# --------------------------

# 1) Linha a linha → 1 apenas nas referências que bateram o limiar
df['ever30m12_snapshot'] = np.where((df['dias_atraso'] >= 30) & (df['idade_meses_contrato'] <= 12),1, 0)

# 2) Contrato inteiro → 1 se em algum mês bateu o limiar
df['ever30m12_painel'] = df.groupby('id_contrato')['ever30m12_snapshot'].transform('max')

df.head(15)


,id_contrato,data_inicio_contrato,safra,data_ref,dias_atraso,flag_acordo,idade_meses_contrato,mau_origem,ever30m6_snapshot,ever30m6_painel,ever30m12_snapshot,ever30m12_painel
0,10000000,2014-09-30,2015-01-01,2015-01-01,30.0000,0,0,1,1,1,1,1
1,10000000,2014-09-30,2015-01-01,2015-02-01,15.0000,0,1,1,0,1,0,1
2,10000000,2014-09-30,2015-01-01,2015-03-01,15.0000,0,2,1,0,1,0,1
3,10000000,2014-09-30,2015-01-01,2015-04-01,30.0000,0,3,1,1,1,1,1
4,10000000,2014-09-30,2015-01-01,2015-05-01,60.0000,0,4,1,1,1,1,1
5,10000000,2014-09-30,2015-01-01,2015-06-01,90.0000,0,5,1,1,1,1,1
6,10000000,2014-09-30,2015-01-01,2015-07-01,60.0000,0,6,1,1,1,1,1
7,10000000,2014-09-30,2015-01-01,2015-08-01,90.0000,0,7,1,0,1,1,1
8,10000000,2014-09-30,2015-01-01,2015-09-01,90.0000,0,8,1,0,1,1,1
9,10000000,2014-09-30,2015-01-01,2015-10-01,60.0000,0,9,1,0,1,1,1


### 1.1.4) Rolagem

In [66]:
# Curva de Performance por Safra - Cohort
bins = [0, 30, 60, 90, 120, float('inf')]
labels = ['1–30 dias','31–60 dias','61–90 dias','91–120 dias','>120 dias']
df['faixa_atraso'] = pd.cut(df['dias_atraso'], bins=bins, labels=labels, right=True)

matriz_cohort = df.groupby(['safra', 'faixa_atraso'])['id_contrato'].count().unstack(fill_value=0)
matriz_cohort_pct = matriz_cohort.div(matriz_cohort.sum(axis=1), axis=0) * 100

matriz_cohort_pct = matriz_cohort_pct.round(2)
display(matriz_cohort_pct)


# Matriz de Rolagem
bins = [0, 30, 60, 90, 120, float('inf')]
labels = ['1-30 dias','31-60 dias','61-90 dias','91-120 dias','>120 dias']
df['faixa_atraso'] = pd.cut(df['dias_atraso'], bins=bins, labels=labels, right=True)

# Ordene por contrato e data
df = df.sort_values(['id_contrato', 'data_ref'])

# Pegue a faixa de atraso do mês anterior
df['faixa_atraso_prev'] = df.groupby('id_contrato')['faixa_atraso'].shift(1)

# Considere apenas pares válidos (com mês anterior)
transicoes = df.dropna(subset=['faixa_atraso_prev', 'faixa_atraso'])

# Monte a matriz de transição
matriz_transicao = pd.crosstab(
    transicoes['faixa_atraso_prev'],
    transicoes['faixa_atraso'],
    normalize='index'
) * 100

display(matriz_transicao)

faixa_atraso,1–30 dias,31–60 dias,61–90 dias,91–120 dias,>120 dias
safra,,,,,
2015-01-01,41.7300,32.0800,16.1300,3.9700,6.1000
2015-02-01,50.6800,29.3300,14.3800,1.7800,3.8300
2015-03-01,49.2000,30.1000,15.2300,1.9600,3.5000
2015-04-01,48.6400,32.2400,14.6200,2.0800,2.4200
2015-05-01,50.8700,30.5400,14.1500,2.0600,2.3800
2015-06-01,48.2800,32.0500,14.9500,2.1500,2.5600
2015-07-01,47.4600,34.1800,13.8400,1.3700,3.1500
2015-08-01,48.4400,32.2300,13.8200,2.0000,3.5200
2015-09-01,51.7500,31.2100,13.3800,1.9000,1.7500


faixa_atraso,1-30 dias,31-60 dias,61-90 dias,91-120 dias,>120 dias
faixa_atraso_prev,,,,,
1-30 dias,84.3684,15.6316,0.0000,0.0000,0.0000
31-60 dias,9.0544,63.5413,27.4043,0.0000,0.0000
61-90 dias,0.0000,51.6292,48.3708,0.0000,0.0000
91-120 dias,0.0000,0.0000,40.2734,59.7266,0.0000
>120 dias,0.0000,0.0000,0.0000,19.6751,80.3249


# <font color='orange' style='font-size: 40px;'> Exemplo Inferência de Negados </font>
<hr style='border: 2px solid orange;'>

https://python.plainenglish.io/a-project-on-reject-inference-94a6858bc821

https://www.kaggle.com/code/shraddhacodes/credit-card-default-reject-inference-project

# <font color='green' style='font-size: 30px;'> 1.1) Teste 1 </font>
<hr style='border: 2px solid green;'>

In [3]:
df_approved = pd.read_excel('./data/appbeh_approved.xlsx')
df_rej = pd.read_excel('./data/appbeh_rej.xlsx')